<a href="https://colab.research.google.com/github/DeependraParichha1004/NLP/blob/main/BertTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 6.4 MB/s 


In [2]:
! pip install -q tensorflow_datasets

In [29]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import time
import pandas as pd
import numpy as np
import pathlib
import re

In [4]:
examples,metadata=tfds.load('ted_hrlr_translate/pt_to_en',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteI7IFMQ/ted_hrlr_translate-train.tfrecord


...

  0%|          | 0/1193 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteI7IFMQ/ted_hrlr_translate-test.tfrecord


  0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples,validation_examples=examples['train'],examples['validation']

In [7]:
type(train_examples)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [8]:
import numpy
for pt,en in train_examples.take(1):
    print(pt.numpy())
    print(en.numpy().decode('utf-8'))

b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'
and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


English Dataset

In [9]:
train_en=train_examples.map(lambda pt,en:en)

In [65]:
train_en

<MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

Hindi Dataset

In [106]:
import pandas as pd
data=pd.read_csv('/content/Hindi_English_Truncated_Corpus.csv')
hin=data['hindi_sentence'].iloc[:].values
hin=tf.data.Dataset.from_tensor_slices(hin)
type(hin)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [96]:
data['hindi_sentence']

0         राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...
1         मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2          यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3            हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4               इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
                                ...                        
127602    आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127603                      और अपने गालों में डाल लेते हैं।
127604    जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द...
127605    Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .
127606    हाल ही में उन्हें सरकारी ठेका मिला है करीब सौ ...
Name: hindi_sentence, Length: 127607, dtype: object

In [103]:
def hin_dataset(ds):
  return (
      ds
      .cache()
      .prefetch(tf.data.AUTOTUNE)
  )

In [107]:
hindi_dataset=hin_dataset(hin)
hindi_dataset

<PrefetchDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [143]:
for hin in hindi_dataset.batch(3).take(1):
  for hn in hin.numpy():
    print(hn.decode('utf-8'))

राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,
यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।


English and Hindi vocab

In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

In [ ]:
bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [16]:
def write_vocab(file_path,en_vocab):
    with open(file_path,'w',encoding='utf-8') as f:
        for words in en_vocab:
            print(words,file=f)

In [73]:
%%time
hin_vocab=bert_vocab.bert_vocab_from_dataset(
hindi_dataset.batch(1000).prefetch(2),
**bert_vocab_args)

CPU times: user 2min 46s, sys: 766 ms, total: 2min 47s
Wall time: 3min


In [74]:
write_vocab('hin_vocab.txt', hin_vocab)

In [15]:
%%time
en_vocab=bert_vocab.bert_vocab_from_dataset(
train_en.batch(500).prefetch(1),
**bert_vocab_args)

CPU times: user 1min 42s, sys: 2.56 s, total: 1min 45s
Wall time: 1min 48s


In [19]:
write_vocab('en_vocab.txt', en_vocab)

## English and Hindi Tokenizer

English Tokenizer

In [123]:
import tensorflow_text as text
en_tokenizer=text.BertTokenizer('en_vocab.txt',**bert_tokenizer_params)
hin_tokenizer=text.BertTokenizer('hin_vocab.txt',**bert_tokenizer_params)

In [21]:
for pt_examples,en_examples in train_examples.batch(3).take(1):
    print(en_examples.numpy())

[b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
 b'but what if it were active ?' b"but they did n't test for curiosity ."]


In [117]:
sequences_eng=en_tokenizer.tokenize(en_examples)
sequences_eng

<tf.RaggedTensor [[[72], [117], [79], [1259], [1491, 2362], [13], [79], [150], [184], [311],
  [71], [103], [2308], [74], [2679], [13], [148], [80],
  [55, 4840, 1434, 2423, 540], [15]]                                       ,
 [[87], [90], [107], [76], [129], [1852], [30]],
 [[87], [83], [149], [50], [9], [56], [664], [85], [2512], [15]]]>

In [118]:
sequences_eng=sequences_eng.merge_dims(1,2)#merge_dims->(word,word-piece)
sequences_eng

<tf.RaggedTensor [[72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74,
  2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15]                       ,
 [87, 90, 107, 76, 129, 1852, 30],
 [87, 83, 149, 50, 9, 56, 664, 85, 2512, 15]]>

Words like "searchability" and "serendipity" <br>have been decomposed into "search ##ability" and "s ##ere ##nd<br>##ip ##ity" using tf.gather.

In [30]:
tokens_eng=tf.gather(en_vocab,sequences_eng)
tokens_eng

<tf.RaggedTensor [[b'and', b'when', b'you', b'improve', b'search', b'##ability', b',',
  b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage',
  b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip',
  b'##ity', b'.']                                                          ,
 [b'but', b'what', b'if', b'it', b'were', b'active', b'?'],
 [b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for', b'curiosity',
  b'.']                                                                    ]>

Hindi Tokenizer

In [132]:
for hn in hindi_dataset.batch(3).take(1):
  for hin in hn.numpy():
    print(hin.decode('utf-8'))

राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,
यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।


In [133]:
sequences_hin=hin_tokenizer.tokenize(hn)
sequences_hin=sequences_hin.merge_dims(-2,-1)
sequences_hin


<tf.RaggedTensor [[1186, 6339, 105, 308, 205, 370, 260, 286, 15, 219, 209, 194, 1049, 197,
  138, 17]                                                               ,
 [1481, 361, 334, 202, 192, 535, 105, 237, 129, 2263, 5997, 15],
 [196, 441, 227, 129, 1849, 441, 137, 254, 138, 150]]>

In [154]:
sent_hin=hin_tokenizer.detokenize(sequences_hin)
sent_hin=tf.strings.reduce_join(sent_hin,separator=' ',axis=-1)
for sent in sent_hin.numpy():
  print(sent.decode('utf-8'))

राजनीतिजञो क पास जो कारय करना चाहिए , वह करन कि अनमति नही ह .
मई आपको ऐस ही एक बचच क बार म बताना चाहगी ,
यह परतिशत भारत म हिनदओ परतिशत स अधिक ह ।


To re-assemble words from the extracted tokens,

In [120]:
sent=en_tokenizer.detokenize(sequences_eng)
tf.strings.reduce_join(sent,separator=' ',axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

Customization and Export

In [158]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

words = en_tokenizer.detokenize(add_start_end(sequences_eng))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] and when you improve searchability , you actually take away the one advantage of print , which is serendipity . [END]',
       b'[START] but what if it were active ? [END]',
       b"[START] but they did n ' t test for curiosity . [END]"],
      dtype=object)>

In [159]:
def clean_up(reserved_tokens,tokens):
    bad_tokens=[re.escape(tok) for tok in reserved_tokens if tok!= '[UNK]']
    bad_tokens_re='|'.join(bad_tokens)
    bad_cells=tf.strings.regex_full_match(tokens,bad_tokens_re)
    # bad_cells=tf.math.equal(tokens,bad_tokens_re)
    result=tf.ragged.boolean_mask(tokens,~bad_cells)
    # result=tf.ragged.boolean_mask(tokens,tf.math.logical_not(bad_cells))
    return tf.strings.reduce_join(result,separator=' ',axis=-1)
words=en_tokenizer.detokenize(sequences_eng)
clean_up(reserved_tokens,words)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

In [160]:

class CustomTokenizer(tf.Module):
    def __init__(self, reserved_tokens, vocab_path):
        
        self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
        self._reserved_tokens = reserved_tokens
        self._vocab_path = tf.saved_model.Asset(vocab_path)

        vocab = pathlib.Path(vocab_path).read_text().splitlines()
        self.vocab = tf.Variable(vocab)

        ## Create the signatures for export:   

        # Include a tokenize signature for a batch of strings. 
        self.tokenize.get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string))

        # Include `detokenize` and `lookup` signatures for:
        #   * `Tensors` with shapes [tokens] and [batch, tokens]
        #   * `RaggedTensors` with shape [batch, tokens]
        self.detokenize.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.detokenize.get_concrete_function(
              tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        self.lookup.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.lookup.get_concrete_function(
              tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        # These `get_*` methods take no arguments
        self.get_vocab_size.get_concrete_function()
        self.get_vocab_path.get_concrete_function()
        self.get_reserved_tokens.get_concrete_function()

    @tf.function
    def tokenize(self, strings):
        enc = self.tokenizer.tokenize(strings)
        # Merge the `word` and `word-piece` axes.
        enc = enc.merge_dims(-2,-1)
        enc = add_start_end(enc)
        return enc

    @tf.function
    def detokenize(self, tokenized):
        words = self.tokenizer.detokenize(tokenized)
        return clean_up(self._reserved_tokens, words)

    @tf.function
    def lookup(self, token_ids):
        return tf.gather(self.vocab, token_ids)

    @tf.function
    def get_vocab_size(self):
        return tf.shape(self.vocab)[0]

    @tf.function
    def get_vocab_path(self):
        return self._vocab_path

    @tf.function
    def get_reserved_tokens(self):
        return tf.constant(self._reserved_tokens)

In [161]:
tokenizers = tf.Module()
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')
tokenizers.hin = CustomTokenizer(reserved_tokens, 'hin_vocab.txt')

In [162]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.saved_model.save(tokenizers, model_name)

INFO:tensorflow:Assets written to: ted_hrlr_translate_pt_en_converter/assets


INFO:tensorflow:Assets written to: ted_hrlr_translate_pt_en_converter/assets


In [163]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.en.get_vocab_size().numpy()

7010

In [164]:
reloaded_tokenizers.hin.get_vocab_size().numpy()

7612

In [166]:
tokens_eng = reloaded_tokenizers.en.tokenize(['Hello TensorFlow!'])
tokens_eng.numpy()

array([[   2, 4006, 2358,  687, 1192, 2365,    4,    3]])

In [168]:
tokens_hin = reloaded_tokenizers.en.tokenize(['Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .'])
tokens_hin.numpy()

array([[ 2,  1, 14,  1,  1,  1,  1,  1,  1,  1,  1,  1, 15,  3]])